# Run Experiments

You can use the Azure Machine Learning SDK to run code experiments that log metrics and generate outputs. This is at the core of most machine learning operations in Azure Machine Learning.

## Connect to your workspace

All experiments and associated resources are managed within your Azure Machine Learning workspace. In most cases, you should store the workspace configuration in a JSON configuration file. This makes it easier to reconnect without needing to remember details like your Azure subscription ID. You can download the JSON configuration file from the blade for your workspace in the Azure portal, but if you're using a Compute Instance within your workspace, the configuration file has already been downloaded to the root folder.

The code below uses the configuration file to connect to your workspace.

> **Note**: If you haven't already established an authenticated session with your Azure subscription, you'll be prompted to authenticate by clicking a link, entering an authentication code, and signing into Azure.

In [ ]:
import azureml.core
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies

from azureml.widgets import RunDetails

In [1]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.28.0 to work with dtu_mlops


In [2]:
import os, shutil

# Create a folder for the experiment files
folder_name = 'fashion-experiment-files'
experiment_folder = './' + folder_name
#os.makedirs(folder_name, exist_ok=True)


Now we'll create a Python script containing the code for our experiment, and save it in the experiment folder.

> **Note**: running the following cell just *creates* the script file - it doesn't run it!

Now you're almost ready to run the experiment. To run the script, you must create a **ScriptRunConfig** that identifies the Python script file to be run in the experiment, and then run an experiment based on it.

> **Note**: The ScriptRunConfig also determines the compute target and Python environment. In this case, the Python environment is defined to include some Conda and pip packages, but the compute target is omitted; so the default local compute will be used.

The following cell configures and submits the script-based experiment.

In [4]:
ws = Workspace.from_config() # This automatically looks for a directory .azureml

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4, 
                                                           idle_seconds_before_scaledown=2400)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
ws = Workspace.from_config()

#'''
# Create a Python environment for the experiment
env = Environment.from_conda_specification(name='Fashion-train',file_path='./conda_dependencies.yml')

experiment = Experiment(workspace=ws, name='Fashion-train')
config = ScriptRunConfig(source_directory='./',
                            script='fashion_trainer.py',
                            compute_target="cpu-cluster",
                            environment=env) 


config.run_config.environment = env

run = experiment.submit(config)
RunDetails(run).show()
run.wait_for_completion()

Submitting /mnt/batch/tasks/shared/LS_root/mounts/clusters/jthorkilsdatter/code/Users/s175385/MLOps/fashion-experiment-files directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'Fashion-train_1623839300_8e71b394',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-06-16T10:28:34.640246Z',
 'endTimeUtc': '2021-06-16T10:30:36.280991Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'a99805b2-6b52-4081-b30a-9935f9b22f32',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'fashion_trainer.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cpu-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'Fashion-train',
   'version': 'Autosave_2021-06-16T09:36:28Z_2ff5

As before, you can use the widget or the link to the experiment in [Azure Machine Learning studio](https://ml.azure.com) to view the outputs generated by the experiment, and you can also write code to retrieve the metrics and files it generated:

In [6]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Accuracy [0.3393999934196472, 0.7983999848365784, 0.8209999799728394, 0.8396000266075134, 0.8522999882698059, 0.8820000290870667, 0.8773000240325928, 0.8944000005722046, 0.899399995803833, 0.9068999886512756]
Loss [2.385221004486084, 0.7372096180915833, 0.7386306524276733, 0.5273603796958923, 0.5369148850440979, 0.36451077461242676, 0.4364103078842163, 0.3081313669681549, 0.42938071489334106, 0.3664677143096924]


azureml-logs/55_azureml-execution-tvmps_8ffa75f2bf9c813bc92d29f90f7c4b90001a000ea8d9520bc28e6fc5247e06af_d.txt
azureml-logs/65_job_prep-tvmps_8ffa75f2bf9c813bc92d29f90f7c4b90001a000ea8d9520bc28e6fc5247e06af_d.txt
azureml-logs/70_driver_log.txt
azureml-logs/75_job_post-tvmps_8ffa75f2bf9c813bc92d29f90f7c4b90001a000ea8d9520bc28e6fc5247e06af_d.txt
azureml-logs/process_info.json
azureml-logs/process_status.json
logs/azureml/93_azureml.log
logs/azureml/job_prep_azureml.log
logs/azureml/job_release_azureml.log
outputs/fashion_model.pkl


## View experiment run history

Now that you've run the same experiment multiple times, you can view the history in [Azure Machine Learning studio](https://ml.azure.com) and explore each logged run. Or you can retrieve an experiment by name from the workspace and iterate through its runs using the SDK:

In [7]:

fashion_experiment = ws.experiments['Fashion-train']
for logged_run in fashion_experiment.get_runs():
    print('Run ID:', logged_run.id)
    metrics = logged_run.get_metrics()
    for key in metrics.keys():
        print('-', key, metrics.get(key))

Run ID: Fashion-train_1623839300_8e71b394
- Accuracy [0.3393999934196472, 0.7983999848365784, 0.8209999799728394, 0.8396000266075134, 0.8522999882698059, 0.8820000290870667, 0.8773000240325928, 0.8944000005722046, 0.899399995803833, 0.9068999886512756]
- Loss [2.385221004486084, 0.7372096180915833, 0.7386306524276733, 0.5273603796958923, 0.5369148850440979, 0.36451077461242676, 0.4364103078842163, 0.3081313669681549, 0.42938071489334106, 0.3664677143096924]
Run ID: Fashion-train_1623838977_d7d363bc
- Accuracy [0.3084999918937683, 0.7699999809265137, 0.8449000120162964, 0.8449000120162964, 0.879800021648407, 0.8672000169754028, 0.8817999958992004, 0.8736000061035156, 0.9054999947547913, 0.8637999892234802]
- Loss [2.341284990310669, 0.6660426259040833, 0.5231822729110718, 0.4844939410686493, 0.4184477627277374, 0.36130577325820923, 0.4177181124687195, 0.3023797869682312, 0.3384348750114441, 0.3038181960582733]
Run ID: Fashion-train_1623838695_5184023f
- Accuracy [0.2379000037908554, 0.7